In [10]:
import os
import sys
import time

import tweepy
import pandas as pd
import numpy as np

In [11]:
import yaml

root_dir = "/Users/gogrean/Documents/kaggle/off_kaggle_ds/elect2016/"
twitter_cred = yaml.load(open(root_dir + 'credentials/twitter.cred'))

auth = tweepy.OAuthHandler(twitter_cred['consumer_token'], 
                           twitter_cred['consumer_secret'])
auth.set_access_token(twitter_cred['access_token_key'], 
                      twitter_cred['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [12]:
with open(root_dir + "data/trumps_followers.txt", 'a') as f:
    for page in tweepy.Cursor(api.followers, screen_name="realDonaldTrump").pages():
        f.writelines([p.screen_name + "\n" for p in page])    

Rate limit reached. Sleeping for: 897
Rate limit reached. Sleeping for: 898
Rate limit reached. Sleeping for: 897


KeyboardInterrupt: 

In [63]:
import numpy as np

screen_names = []
with open(root_dir + "data/trumps_followers.txt", 'r') as f:
    for screen_name in f.readlines():
        screen_names.append(screen_name.strip())
sorted(np.unique(screen_names))

['008snoussisalim',
 '0102708233',
 '01Bruceman',
 '042niga',
 '06AdnanAkdag',
 '07038279656y',
 '0986a513a90a460',
 '0_omustafa',
 '0megaCast',
 '1012598132Z',
 '101Puppies1',
 '10FootDwarf',
 '10sShell',
 '11sajo',
 '12017019082m',
 '1244gj',
 '141Dgm',
 '14ENG88',
 '150297Sa',
 '1731Hohneck',
 '17amcnamara',
 '17f6efdc9163444',
 '17willsellers00',
 '1809smile',
 '1978Deebo',
 '1980monkeyman',
 '1Hunterodouglas',
 '1KNate',
 '1Mamonchik',
 '1Menfis',
 '1RicardoSerrano',
 '1Satirist',
 '1_stonner',
 '1ceebdafb9fc439',
 '1jamaicanson',
 '2001ECS',
 '2005gapp',
 '2016Roly',
 '201G30F16',
 '213JustDoRight',
 '21Soares715',
 '2252Abo',
 '239gchs',
 '26a1278bc87442e',
 '27Leivin',
 '2rites',
 '307314388Da',
 '30A6VAKkvd1LW5s',
 '30Gpj0pNjzUYjPj',
 '32cvg',
 '334_7918090',
 '36912_rim',
 '3Struck',
 '3d6a4a980c6847e',
 '3napak',
 '3tearsfisher',
 '3xaGranny',
 '4396bear1',
 '47rainier',
 '4801820103Beene',
 '4918eee94be946b',
 '4siteadvisors',
 '507Durvanpty',
 '541094032',
 '5_rjrasmussen'